<a href="https://colab.research.google.com/github/julianasf-91/fusionAI/blob/main/Agente_de_VR_Mensal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agente de VR Mensal

## Instalação e importação de bibliotecas

In [1]:
# Instalar bibliotecas
!pip install --quiet langchain openai pandas langchain_community langchain_experimental azure-identity azure-keyvault-secrets azure-storage-blob pandas numpy openpyxl xlsxwriter langchain langchain-openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.9/187.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.9/412.9 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50

In [2]:
# Importar bibliotecas
import pandas as pd
import zipfile
from io import BytesIO, TextIOWrapper
from google.colab import files
import requests
from langchain_community.chat_models import ChatOpenAI
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
import os
from azure.identity import DefaultAzureCredential, AzureCliCredential
from azure.keyvault.secrets import SecretClient
from langchain.schema import SystemMessage
import numpy as np
from datetime import datetime
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain.prompts import PromptTemplate
from langchain.tools import Tool

## Importação de API Key da Open AI

In [3]:
# Configuração do Azure Key Vault
key_vault_name = "integracaoapi"  # Substitua pelo nome do seu Key Vault
key_vault_url = f"https://{key_vault_name}.vault.azure.net/"
secret_name = "apiopenia"  # Nome do segredo no Key Vault que armazena a chave da OpenAI

# Configurar variáveis de ambiente para o Service Principal
os.environ["AZURE_CLIENT_ID"] = "def49b6b-d37b-412c-979b-e77f0e799e6b"  # Substitua pelo Application (client) ID
os.environ["AZURE_CLIENT_SECRET"] = "c-v8Q~6MGasbu5x8QrpeYYsj2D2uLCZm14Z4baWm"  # Substitua pelo Client Secret
os.environ["AZURE_TENANT_ID"] = "b09ec21a-88ae-42ec-8f20-5bc07b2a1132"  # Substitua pelo Directory (tenant) ID

# Autenticação com Azure
credential = DefaultAzureCredential()
secret_client = SecretClient(vault_url=key_vault_url, credential=credential)

# Recuperar a chave da OpenAI do Key Vault
openai_api_key = secret_client.get_secret(secret_name).value
os.environ["OPENAI_API_KEY"] = openai_api_key

## Upload da pasta .zip


In [4]:
# Upload do ZIP
print("Faça upload do arquivo de dados (ex: Desafio 4 - Dados.zip)")
uploaded = files.upload()
zip_path = list(uploaded.keys())[0]
output_path = "VR_MENSAL_05_2025.xlsx"

Faça upload do arquivo de dados (ex: Desafio 4 - Dados.zip)


Saving Desafio 4 - Dados.zip to Desafio 4 - Dados.zip


## Cálculos e identificação de UF


In [5]:
# Dicionário de Estados e UF
UF_TO_ESTADO = {
    "SP": "São Paulo",
    "RJ": "Rio de Janeiro",
    "RS": "Rio Grande do Sul",
    "PR": "Paraná",
}
PERIOD_START = pd.Timestamp(2025, 4, 15)
PERIOD_END   = pd.Timestamp(2025, 5, 15)
COMPETENCIA  = pd.Timestamp(2025, 5, 1)

# Cálculo de dias
def business_days(start, end):
    return np.busday_count(start.date(), end.date())

def proportion_by_busdays(cut, start, end):
    cut = min(max(cut, start), end)
    total = business_days(start, end)
    if total == 0: return 0.0
    part = business_days(start, cut)
    return min(max(part/total, 0.0), 1.0)

# Extraçao de UF
def extract_uf_from_sindicato(sindicato):
    if not isinstance(sindicato, str): return None
    s = sindicato.upper()
    for uf in UF_TO_ESTADO.keys():
        if f" {uf} " in s or s.endswith(f" {uf}") or f"- {uf}" in s:
            return uf
    for t in s.replace("-", " ").split():
        if t in UF_TO_ESTADO:
            return t
    return None

## Definir Agentes

In [6]:
state = {}

def find_file(z, patterns):
    """
    Busca o primeiro arquivo no zip cujo nome contenha algum dos padrões.
    """
    for fname in z.namelist():
        upper = fname.upper()
        for p in patterns:
            if p in upper:
                return fname
    raise FileNotFoundError(f"Nenhum arquivo encontrado para padrões: {patterns}")

def agente_ingestao(_: str) -> str:
    with zipfile.ZipFile(BytesIO(uploaded[zip_path]), "r") as z:
        ativos = pd.read_excel(z.open(find_file(z, ["ATIVOS"])))
        ativos["MATRICULA"] = pd.to_numeric(ativos["MATRICULA"], errors="coerce").astype("Int64")
        state["ativos"] = ativos

        adm = pd.read_excel(z.open(find_file(z, ["ADMISSAO", "ADMISS", "ADMISSÃO"])))
        adm = adm.rename(columns={adm.columns[0]: "MATRICULA", adm.columns[1]: "Admissao"})
        adm["MATRICULA"] = pd.to_numeric(adm["MATRICULA"], errors="coerce").astype("Int64")
        adm["Admissao"] = pd.to_datetime(adm["Admissao"], errors="coerce")
        state["adm"] = adm

        deslig = pd.read_excel(z.open(find_file(z, ["DESLIG"])))
        deslig = deslig.rename(columns={deslig.columns[0]: "MATRICULA", deslig.columns[1]: "DATA_DEMISSAO", deslig.columns[2]: "COMUNICADO"})
        deslig["MATRICULA"] = pd.to_numeric(deslig["MATRICULA"], errors="coerce").astype("Int64")
        deslig["DATA_DEMISSAO"] = pd.to_datetime(deslig["DATA_DEMISSAO"], errors="coerce")
        state["deslig"] = deslig

        ferias = pd.read_excel(z.open(find_file(z, ["FERIAS", "FÉRIAS"])))
        ferias = ferias.rename(columns={ferias.columns[0]: "MATRICULA", ferias.columns[1]: "DIAS_FERIAS"})
        ferias["MATRICULA"] = pd.to_numeric(ferias["MATRICULA"], errors="coerce").astype("Int64")
        ferias["DIAS_FERIAS"] = pd.to_numeric(ferias["DIAS_FERIAS"], errors="coerce").fillna(0).astype(int)
        state["ferias"] = ferias

        dias = pd.read_excel(z.open(find_file(z, ["DIAS UTEIS"])))
        dias = dias.rename(columns={dias.columns[0]: "Sindicato", dias.columns[1]: "DIAS_UTEIS_TABELA"})
        state["dias"] = dias

        valores = pd.read_excel(z.open(find_file(z, ["VALOR", "SINDICATO"])))
        valores = valores.rename(columns={valores.columns[0]: "ESTADO", valores.columns[1]: "VALOR_DIARIO_VR"})
        state["valores"] = valores

    return "[OK] Bases ingeridas."

def agente_limpeza(_: str) -> str:
    base = state["ativos"].copy()
    base["UF"] = base["Sindicato"].apply(extract_uf_from_sindicato)
    base["ESTADO"] = base["UF"].map(UF_TO_ESTADO)
    base = base.merge(state["dias"], on="Sindicato", how="left")
    base = base.merge(state["valores"], on="ESTADO", how="left")
    base = base.merge(state["ferias"], on="MATRICULA", how="left")
    base = base.merge(state["adm"], on="MATRICULA", how="left")
    base = base.merge(state["deslig"], on="MATRICULA", how="left")
    state["base"] = base
    return "[OK] Base validada e consolidada."

def agente_calculo(_: str) -> str:
    base = state["base"].copy()

    def prop_admissao(adm_date):
        if pd.isna(adm_date): return 1.0
        start = max(PERIOD_START, pd.Timestamp(adm_date))
        if start >= PERIOD_END: return 0.0
        return proportion_by_busdays(start, PERIOD_START, PERIOD_END)

    def prop_deslig(row):
        dem, com = row["DATA_DEMISSAO"], str(row.get("COMUNICADO", "")).upper()
        if pd.isna(dem): return 1.0
        if com == "OK" and dem <= PERIOD_END:
            return 0.0
        cut = min(pd.Timestamp(dem), PERIOD_END)
        return proportion_by_busdays(cut, PERIOD_START, PERIOD_END)

    base["PROP_ADMISSAO"] = base["Admissao"].apply(prop_admissao)
    base["PROP_DESLIG"] = base.apply(prop_deslig, axis=1)
    base["DIAS_AJUSTADOS"] = base["DIAS_UTEIS_TABELA"] * base["PROP_ADMISSAO"] * base["PROP_DESLIG"]
    base["DIAS_FINAIS"] = (base["DIAS_AJUSTADOS"] - base["DIAS_FERIAS"].fillna(0)).clip(lower=0).astype(int)
    base["TOTAL"] = (base["DIAS_FINAIS"] * base["VALOR_DIARIO_VR"]).round(2)
    base["CUSTO_EMPRESA"] = (base["TOTAL"] * 0.80).round(2)
    base["DESC_FUNCIONARIO"] = (base["TOTAL"] * 0.20).round(2)
    state["calc"] = base
    return f"[OK] Cálculo concluído para {len(base)} colaboradores."

def agente_saida(_: str) -> str:
    base = state["calc"]
    out = pd.DataFrame({
        "Matricula": base["MATRICULA"],
        "Admissão": base["Admissao"],
        "Sindicato do Colaborador": base["Sindicato"],
        "Competência": COMPETENCIA,
        "Dias": base["DIAS_FINAIS"],
        "VALOR DIÁRIO VR": base["VALOR_DIARIO_VR"],
        "TOTAL": base["TOTAL"],
        "Custo empresa": base["CUSTO_EMPRESA"],
        "Desconto profissional": base["DESC_FUNCIONARIO"],
        "OBS GERAL": "",
    })
    out.to_excel(output_path, index=False)
    state["out"] = out
    return f"[OK] Arquivo gerado: {output_path}"

## Orquestração de Tools

In [7]:
# Supervisor com LangChain
tools = [
    Tool(name="Ingestao", func=agente_ingestao, description="Carrega e normaliza as bases do zip."),
    Tool(name="LimpezaValidacao", func=agente_limpeza, description="Aplica exclusões e junta dados auxiliares."),
    Tool(name="Calculo", func=agente_calculo, description="Calcula dias e valores de VR."),
    Tool(name="Saida", func=agente_saida, description="Gera o Excel final."),
]

from langchain import hub

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# pega o prompt ReAct oficial do Hub
prompt = hub.pull("hwchase17/react")

agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)

executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=6
)

# Executar Supervisor
# ==============================
executor.invoke({"input": "Execute o pipeline completo até gerar o XLSX final."})

/usr/local/lib/python3.12/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...
Para executar o pipeline completo, preciso seguir as etapas de ingestão, limpeza e validação, cálculo e, finalmente, gerar o arquivo Excel. Vou começar pela ingestão dos dados.

Action: Ingestao  
Action Input: "dados.zip"  [OK] Bases ingeridas.Agora que as bases foram ingeridas, o próximo passo é aplicar a limpeza e validação dos dados para garantir que estejam prontos para o cálculo.

Action: LimpezaValidacao  
Action Input: "dados"  [OK] Base validada e consolidada.Agora que a base de dados foi validada e consolidada, o próximo passo é realizar os cálculos necessários para determinar os dias e valores de VR.

Action: Calculo  
Action Input: "dados"  [OK] Cálculo concluído para 1815 colaboradores.Agora que os cálculos foram concluídos para os colaboradores, o próximo passo é gerar o arquivo Excel final com os resultados.

Action: Saida  
Action Input: "dados"  [OK] Arquivo gerado: VR_MENSAL_05_2025.xlsxI now know the final answer  
Final Answer

{'input': 'Execute o pipeline completo até gerar o XLSX final.',
 'output': 'O pipeline completo foi executado e o arquivo Excel final gerado é: VR_MENSAL_05_2025.xlsx.'}

## Resultado

In [8]:
# ler arquivo VR_MENSAL_05_2025.XLSX
VR_MENSAL_05_2025 = pd.read_excel("VR_MENSAL_05_2025.xlsx")
VR_MENSAL_05_2025

,Matricula,Admissão,Sindicato do Colaborador,Competência,Dias,VALOR DIÁRIO VR,TOTAL,Custo empresa,Desconto profissional,OBS GERAL
0,34941,NaT,SINDPD SP - SIND.TRAB.EM PROC DADOS E EMPR.EMP...,2025-05-01,22,37.5,825.0,660,165.0,NaN
1,24401,NaT,SINDPPD RS - SINDICATO DOS TRAB. EM PROC. DE D...,2025-05-01,21,35.0,735.0,588,147.0,NaN
2,32104,NaT,SINDPPD RS - SINDICATO DOS TRAB. EM PROC. DE D...,2025-05-01,21,35.0,735.0,588,147.0,NaN
3,35254,NaT,SINDPPD RS - SINDICATO DOS TRAB. EM PROC. DE D...,2025-05-01,21,35.0,735.0,588,147.0,NaN
4,31731,NaT,SINDPPD RS - SINDICATO DOS TRAB. EM PROC. DE D...,2025-05-01,21,35.0,735.0,588,147.0,NaN
...,...,...,...,...,...,...,...,...,...,...
1810,32050,NaT,SINDPPD RS - SINDICATO DOS TRAB. EM PROC. DE D...,2025-05-01,21,35.0,735.0,588,147.0,NaN
1811,32549,NaT,SINDPPD RS - SINDICATO DOS TRAB. EM PROC. DE D...,2025-05-01,21,35.0,735.0,588,147.0,NaN
1812,31372,NaT,SINDPPD RS - SINDICATO DOS TRAB. EM PROC. DE D...,2025-05-01,21,35.0,735.0,588,147.0,NaN
1813,31573,NaT,SINDPPD RS - SINDICATO DOS TRAB. EM PROC. DE D...,2025-05-01,21,35.0,735.0,588,147.0,NaN
